In [37]:
# Import our libraries 

# Pandas and numpy for data wrangling
import pandas as pd
import numpy as np

# Seaborn / matplotlib for visualization 
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline

# Import the trees from sklearn
from sklearn import tree

# Helper function to split our data
from sklearn.model_selection import train_test_split

# Helper fuctions to evaluate our model.
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score 

# Helper function for hyper-parameter turning.
from sklearn.model_selection import GridSearchCV

# Import our Decision Tree
from sklearn.tree import DecisionTreeClassifier 

# Import our Random Forest 
from sklearn.ensemble import RandomForestClassifier

# Library for visualizing our tree
# If you get an error, run 'conda install python-graphviz' in your terminal
import graphviz



In [38]:
df = pd.read_csv('Project Dataset/previous_application.csv')
df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# check the shape of the dataset
df.shape

(1670214, 37)

In [40]:
# check for null
df.isnull().sum()

SK_ID_PREV                           0
SK_ID_CURR                           0
NAME_CONTRACT_TYPE                   0
AMT_ANNUITY                     372235
AMT_APPLICATION                      0
AMT_CREDIT                           1
AMT_DOWN_PAYMENT                895844
AMT_GOODS_PRICE                 385515
WEEKDAY_APPR_PROCESS_START           0
HOUR_APPR_PROCESS_START              0
FLAG_LAST_APPL_PER_CONTRACT          0
NFLAG_LAST_APPL_IN_DAY               0
RATE_DOWN_PAYMENT               895844
RATE_INTEREST_PRIMARY          1664263
RATE_INTEREST_PRIVILEGED       1664263
NAME_CASH_LOAN_PURPOSE               0
NAME_CONTRACT_STATUS                 0
DAYS_DECISION                        0
NAME_PAYMENT_TYPE                    0
CODE_REJECT_REASON                   0
NAME_TYPE_SUITE                 820405
NAME_CLIENT_TYPE                     0
NAME_GOODS_CATEGORY                  0
NAME_PORTFOLIO                       0
NAME_PRODUCT_TYPE                    0
CHANNEL_TYPE             

In [41]:
# check how many duplicated row in the dataset
df.duplicated().sum()

0

In [42]:
# drop the cancled loan application cuz it does not make any contribution to our reseach topic
# and mark Approved and Unused offers as approved since they got accepted
# and mark Refused loan as 0
df['NAME_CONTRACT_STATUS']=df['NAME_CONTRACT_STATUS'].replace('Approved', 1)
df['NAME_CONTRACT_STATUS']=df['NAME_CONTRACT_STATUS'].replace('Refused', 0)
df['NAME_CONTRACT_STATUS']=df['NAME_CONTRACT_STATUS'].replace('Unused offer', 1)
df.drop(df[df['NAME_CONTRACT_STATUS'] =='Canceled'].index, inplace = True)


In [43]:
#since we are not yet interested in the reject reason, drop it for now
df.drop('CODE_REJECT_REASON', axis=1, inplace=True)
# we are not interested in how approved loan is paid back now, drop it
df.drop('NAME_PAYMENT_TYPE', axis=1, inplace=True)

df.drop(['SELLERPLACE_AREA','NAME_SELLER_INDUSTRY','CNT_PAYMENT','DAYS_FIRST_DRAWING','DAYS_FIRST_DUE'
        ,'DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION'], axis=1, inplace=True)

In [44]:
# check if the Canceled loan still exist
df['NAME_CONTRACT_STATUS'].value_counts()

1    1063217
0     290678
Name: NAME_CONTRACT_STATUS, dtype: int64

In [45]:
# check the shape of the dataset again, obviously, the canceled data is dropped
df.shape

(1353895, 27)

In [9]:
# convert text type data into numerical data
df = pd.get_dummies(df, columns=['NAME_CONTRACT_TYPE','WEEKDAY_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT',
                                 'NAME_CASH_LOAN_PURPOSE','NAME_TYPE_SUITE','NAME_CLIENT_TYPE',
                                'NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE',
                                'CHANNEL_TYPE','NAME_YIELD_GROUP','PRODUCT_COMBINATION','NFLAG_INSURED_ON_APPROVAL'], drop_first=True)

In [46]:
df.isnull().sum()

SK_ID_PREV                           0
SK_ID_CURR                           0
NAME_CONTRACT_TYPE                   0
AMT_ANNUITY                      66430
AMT_APPLICATION                      0
AMT_CREDIT                           1
AMT_DOWN_PAYMENT                580168
AMT_GOODS_PRICE                  79945
WEEKDAY_APPR_PROCESS_START           0
HOUR_APPR_PROCESS_START              0
FLAG_LAST_APPL_PER_CONTRACT          0
NFLAG_LAST_APPL_IN_DAY               0
RATE_DOWN_PAYMENT               580168
RATE_INTEREST_PRIMARY          1347944
RATE_INTEREST_PRIVILEGED       1347944
NAME_CASH_LOAN_PURPOSE               0
NAME_CONTRACT_STATUS                 0
DAYS_DECISION                        0
NAME_TYPE_SUITE                 514066
NAME_CLIENT_TYPE                     0
NAME_GOODS_CATEGORY                  0
NAME_PORTFOLIO                       0
NAME_PRODUCT_TYPE                    0
CHANNEL_TYPE                         0
NAME_YIELD_GROUP                     0
PRODUCT_COMBINATION      

In [48]:
df.head(20)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,DAYS_DECISION,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_YIELD_GROUP,PRODUCT_COMBINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,-73,NaN,Repeater,Mobile,POS,XNA,Country-wide,middle,POS mobile with interest,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,-164,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,low_action,Cash X-Sell: low,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,-301,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,high,Cash X-Sell: high,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,-512,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,middle,Cash X-Sell: middle,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,-781,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,high,Cash Street: high,NaN
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,...,-684,Family,Repeater,XNA,Cash,x-sell,Credit and cash offices,low_normal,Cash X-Sell: low,1.0
10,1715995,447712,Cash loans,11368.620,270000.0,335754.0,NaN,270000.0,FRIDAY,7,...,-735,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,low_normal,Cash X-Sell: low,1.0
11,2257824,161140,Cash loans,13832.775,211500.0,246397.5,NaN,211500.0,FRIDAY,10,...,-815,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,middle,Cash X-Sell: middle,1.0
12,2330894,258628,Cash loans,12165.210,148500.0,174361.5,NaN,148500.0,TUESDAY,15,...,-860,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,high,Cash X-Sell: high,1.0
13,1397919,321676,Consumer loans,7654.860,53779.5,57564.0,0.0,53779.5,SUNDAY,15,...,-408,Unaccompanied,New,Consumer Electronics,POS,XNA,Country-wide,low_action,POS household without interest,1.0


In [ ]:
# get the average annuity of approved applications and refused applications
average_approved_AMT_ANNUITY=(df[(df['NAME_CONTRACT_STATUS']==1)])['AMT_ANNUITY'].mean()
average_Refused_ANNUITY=(df[(df['NAME_CONTRACT_STATUS']==0)])['AMT_ANNUITY'].mean()
print("the average annuity of approved applications is ", average_approved_AMT_ANNUITY)
print("the average annuity of refused applications is ", average_Refused_ANNUITY)

In [ ]:
#it seems like we have to fill something into the null values
#some AMT_ANNUITY is empty, I will replace the the average AMT_ANNUITY based on its NAME_CONTRACT_STATUS

df['AMT_ANNUITY'] = np.where(((df['AMT_ANNUITY']=='nan') & (df['NAME_CONTRACT_STATUS'] == 1) ), average_approved_AMT_ANNUITY,df['AMT_ANNUITY'] )
df['AMT_ANNUITY'] = np.where(((df['AMT_ANNUITY']=='nan') & (df['NAME_CONTRACT_STATUS'] == 0) ), average_Refused_ANNUITY,df['AMT_ANNUITY'] )

In [64]:
df.head(50)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,DAYS_DECISION,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_YIELD_GROUP,PRODUCT_COMBINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.43,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,-73,NaN,Repeater,Mobile,POS,XNA,Country-wide,middle,POS mobile with interest,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,-164,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,low_action,Cash X-Sell: low,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,-301,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,high,Cash X-Sell: high,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,-512,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,middle,Cash X-Sell: middle,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,-781,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,high,Cash Street: high,NaN
5,1383531,199383,Cash loans,23703.93,315000.0,340573.5,NaN,315000.0,SATURDAY,8,...,-684,Family,Repeater,XNA,Cash,x-sell,Credit and cash offices,low_normal,Cash X-Sell: low,1.0
10,1715995,447712,Cash loans,11368.62,270000.0,335754.0,NaN,270000.0,FRIDAY,7,...,-735,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,low_normal,Cash X-Sell: low,1.0
11,2257824,161140,Cash loans,13832.775,211500.0,246397.5,NaN,211500.0,FRIDAY,10,...,-815,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,middle,Cash X-Sell: middle,1.0
12,2330894,258628,Cash loans,12165.21,148500.0,174361.5,NaN,148500.0,TUESDAY,15,...,-860,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,high,Cash X-Sell: high,1.0
13,1397919,321676,Consumer loans,7654.86,53779.5,57564.0,0.0,53779.5,SUNDAY,15,...,-408,Unaccompanied,New,Consumer Electronics,POS,XNA,Country-wide,low_action,POS household without interest,1.0
